# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import pickle
import warnings
warnings.filterwarnings('ignore') 

In [2]:
# download necessary NLTK data
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster_Project.db')
df = pd.read_sql_table('dis_data_prepro',engine)
 
# Extract X and y variables from the data for the modelling
X = df['message']
y = df.iloc[:,4:]

category_names = list(df.columns[4:])

In [4]:
# check of data types
df.dtypes

id                         int64
message                   object
original                  object
genre                     object
related                    int64
request                    int64
offer                      int64
aid_related                int64
medical_help               int64
medical_products           int64
search_and_rescue          int64
security                   int64
military                   int64
child_alone                int64
water                      int64
food                       int64
shelter                    int64
clothing                   int64
money                      int64
missing_people             int64
refugees                   int64
death                      int64
other_aid                  int64
infrastructure_related     int64
transport                  int64
buildings                  int64
electricity                int64
tools                      int64
hospitals                  int64
shops                      int64
aid_center

In [5]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

In [6]:
y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
   
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [5]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Pipeline uses DecisionTree Classifier
pipelineDTC = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(DecisionTreeClassifier())) 
    ])

In [7]:
# Pipeline uses KNeighbors Classifier along with a custom made transformer (StartingVerbExtractor)
pipelineKN = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier()))
    ])

In [8]:
# Pipeline uses Random Forest Classifier
pipelineRF = Pipeline([
         ('vect', CountVectorizer(tokenizer=tokenize)),
         ('tfidf', TfidfTransformer()),
         ('clf',  MultiOutputClassifier(RandomForestClassifier(random_state=42)))
     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
    
#https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

#https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [13]:
pipelineDTC_fitted = pipelineDTC.fit(X_train, y_train)

In [10]:
pipelineKN_fitted = pipelineKN.fit(X_train, y_train)

In [26]:
pipelineRF_fitted = pipelineRF.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
%%time
y_predKN = pipelineKN_fitted.predict(X_test)
# Print classification report on test data    
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print(classification_report(y_test.iloc[:, i].values, y_predKN[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_predKN[:, i])))

Category: related 
             precision    recall  f1-score   support

          0       0.33      0.15      0.20       855
          1       0.65      0.85      0.74      1666
          2       0.00      0.00      0.00        35

avg / total       0.53      0.60      0.55      2556

Accuracy 0.6025039123630673


Category: request 
             precision    recall  f1-score   support

          0       0.67      0.87      0.76      1683
          1       0.42      0.19      0.26       873

avg / total       0.59      0.63      0.59      2556

Accuracy 0.6341940532081377


Category: offer 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2554
          1       0.00      0.00      0.00         2

avg / total       1.00      1.00      1.00      2556

Accuracy 0.9992175273865415


Category: aid_related 
             precision    recall  f1-score   support

          0       0.62      0.82      0.71      1569
          1       0.43     

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipelineKN.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__starting_verb_transformer', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'feat

In [16]:
#param_grid = {'vect__min_df': [1, 5],
#              'tfidf__use_idf':[True, False],
#              'clf__estimator__min_samples_split':[2, 5, 10]}

param_gridKN = {'clf__estimator__n_neighbors': (1,10),
                'clf__estimator__leaf_size': (20,1)}
                

#CV_KN = GridSearchCV(estimator=pipelineKN, param_grid=param_grid)
CV_KN = GridSearchCV(estimator=pipelineKN, param_grid=param_gridKN)
CV_KN.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_neighbors': (1, 10), 'clf__estimator__leaf_size': (20, 1)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [17]:
print("The best hyperparameters from Grid Search are:")
print(CV_KN.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(CV_KN.best_score_)

The best hyperparameters from Grid Search are:
{'clf__estimator__leaf_size': 20, 'clf__estimator__n_neighbors': 10}

The mean accuracy of a model with these hyperparameters is:
0.187035346289


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
from sklearn.metrics import classification_report
# Predicting using the first tuned model 
y_pred = CV_KN.predict(X_train)

# Training accuracy
print("Training Data: ")
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print(classification_report(y_train.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_train.iloc[:, i].values, y_pred[:, i])))

Training Data: 
Category: related 
             precision    recall  f1-score   support

          0       0.60      0.31      0.41      2521
          1       0.71      0.90      0.80      5052
          2       0.83      0.05      0.10        94

avg / total       0.68      0.70      0.66      7667

Accuracy 0.6950567366636233


Category: request 
             precision    recall  f1-score   support

          0       0.68      0.96      0.79      4944
          1       0.69      0.17      0.27      2723

avg / total       0.68      0.68      0.61      7667

Accuracy 0.6778400939089605


Category: offer 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7659
          1       0.00      0.00      0.00         8

avg / total       1.00      1.00      1.00      7667

Accuracy 0.998956567105778


Category: aid_related 
             precision    recall  f1-score   support

          0       0.66      0.94      0.77      4707
          1 

In [20]:
y_pred = CV_KN.predict(X_test)

# Test accuracy
print("Test Data: ")
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))

Test Data: 
Category: related 
             precision    recall  f1-score   support

          0       0.39      0.06      0.11       855
          1       0.66      0.95      0.78      1666
          2       0.00      0.00      0.00        35

avg / total       0.56      0.64      0.54      2556

Accuracy 0.6412363067292645


Category: request 
             precision    recall  f1-score   support

          0       0.67      0.92      0.77      1683
          1       0.44      0.12      0.19       873

avg / total       0.59      0.65      0.58      2556

Accuracy 0.6474960876369327


Category: offer 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2554
          1       0.00      0.00      0.00         2

avg / total       1.00      1.00      1.00      2556

Accuracy 0.9992175273865415


Category: aid_related 
             precision    recall  f1-score   support

          0       0.62      0.86      0.73      1569
          1    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
#pipelineTFTLSVMC_fitted = pipelineTFTLSVMC.fit(X_train, y_train)
pipelineDTC_fitted = pipelineDTC.fit(X_train, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: False

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.